In [1]:
from IPython.core.display import HTML
with open ("../style.css", "r") as file:
    css = file.read()
HTML(css)

# Dealing with Conflicts

This file shows how `Ply` deals with *shift/reduce* and *reduce/reduce* conflicts.

## Specification of the Scanner

We implement a minimal scanner for arithmetic expressions.

In [2]:
import ply.lex as lex

tokens = [ 'NUMBER' ]

def t_NUMBER(t):
    r'0|[1-9][0-9]*'
    t.value = float(t.value)
    return t

literals = ['+', '-', '*', '/', '(', ')']

t_ignore  = ' \t'

def t_newline(t):
    r'\n+'
    t.lexer.lineno += t.value.count('\n')

def t_error(t):
    print(f"Illegal character '{t.value[0]}'")
    t.lexer.skip(1)

__file__ = 'main'

lexer = lex.lex()

## Specification of the Parser

In [3]:
import ply.yacc as yacc

The *start variable* of our grammar is `expr`, but we dont't have to specify that.  The default
start variable is the first vvariable that is defined.

In [4]:
start = 'expr'

We can specify multiple expressions in a single rule.  In this case, we have used the `pass`statement 
as we just want to generate some conflicts. 

It is <b>very important</b> that in the grammar rules below the `:` is preceded by a space!

In [5]:
def p_expr(p):
    """
    expr : expr '+' expr
         | expr '-' expr
         | expr '*' expr
         | expr '/' expr
         | '(' expr ')'
         | NUMBER            
    """
    pass

In [6]:
def p_error(p):
    if p:
        print(f'Syntax error at {p.value}.')
    else:
        print('Syntax error at end of input.')

Setting the optional argument `write_tables` to `False` <B style="color:red">is required</B> to prevent an obscure bug where the parser generator tries  to read an empty parse table.

In [7]:
parser = yacc.yacc(write_tables=False, debug=True)

Generating LALR tables


Let's look at the action table that is generated.  Conflicts are always resolved in favour of shifting.

In [8]:
!cat parser.out

Created by PLY version 3.11 (http://www.dabeaz.com/ply)

Grammar

Rule 0     S' -> expr
Rule 1     expr -> expr + expr
Rule 2     expr -> expr - expr
Rule 3     expr -> expr * expr
Rule 4     expr -> expr / expr
Rule 5     expr -> ( expr )
Rule 6     expr -> NUMBER

Terminals, with rules where they appear

(                    : 5
)                    : 5
*                    : 3
+                    : 1
-                    : 2
/                    : 4
NUMBER               : 6
error                : 

Nonterminals, with rules where they appear

expr                 : 1 1 2 2 3 3 4 4 5 0

Parsing method: LALR

state 0

    (0) S' -> . expr
    (1) expr -> . expr + expr
    (2) expr -> . expr - expr
    (3) expr -> . expr * expr
    (4) expr -> . expr / expr
    (5) expr -> . ( expr )
    (6) expr -> . NUMBER

    (               shift and go to state 2
    NUMBER          shift and go to state 3

    expr                           shift and go to state 1

state 1

    (0) S' -> expr .
